# **Feature Engineering**

In this section features will be created to enhance the performance of machine learning models. This involves transforming raw data into meaningful representations that can better capture the underlying patterns and relationships.

**Install RD-Kit.**

RD-Kit is an open-source Python package for data scientists to work with chemistry data.

In [ ]:
! pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 11.5 MB/s eta 0:00:00


**Install the other required packages.**

In [ ]:
! pip install py3Dmol

In [ ]:
! pip install mol2vec gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00


**Import necessary libraries and modules.**

In [ ]:
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
import py3Dmol
from ipywidgets import interact,fixed,IntSlider
import ipywidgets
from rdkit.Chem import Crippen
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles
from mol2vec.features import mol2alt_sentence, MolSentence
import gensim
import pickle
from mol2vec.features import mol2sentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
from gensim.models import Word2Vec

*Molecular weight.*

In [ ]:
# Define a function to extract the first number from a string
def extract_MW(s):
    return float(s.split()[0])

# Apply the function to the entire DataFrame
df['MW'] = df['FW'].apply(extract_MW)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11759 entries, 0 to 11763
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Formula        11759 non-null  object 
 1   FW             11759 non-null  object 
 2   DSSTox_CID     11759 non-null  object 
 3   SR-HSE         8147 non-null   object 
 4   ID             11759 non-null  object 
 5   ROMol          11759 non-null  object 
 6   NR-AR          9358 non-null   object 
 7   SR-ARE         7165 non-null   object 
 8   NR-Aromatase   7222 non-null   object 
 9   NR-ER-LBD      8749 non-null   object 
 10  NR-AhR         8165 non-null   object 
 11  SR-MMP         7317 non-null   object 
 12  NR-ER          7694 non-null   object 
 13  NR-PPAR-gamma  8180 non-null   object 
 14  SR-p53         8630 non-null   object 
 15  SR-ATAD5       9087 non-null   object 
 16  NR-AR-LBD      8595 non-null   object 
 17  SMILES         11759 non-null  object 
 18  MW         

*Structural and physical properties.*

In [ ]:
# Calculating lipophilicity

df['LogP'] = df['ROMol'].apply(lambda x: Crippen.MolLogP(x) if x is not None else None)

# Calculating Polar Surface Area (PSA)

df['PSA'] = df['ROMol'].apply(lambda x: rdMolDescriptors.CalcTPSA(x) if x is not None else None)

# Number of heavy atoms
df['num_of_heavy_atoms'] = df['ROMol'].apply(lambda x: x.GetNumHeavyAtoms() if x is not None else None)  # GetNumHeavyAtoms() method returns a nubmer of all atoms in a molecule with molecular weight > 1

# Number of heteroatoms
df['num_heteroatoms'] = df['ROMol'].apply(lambda x: Descriptors.NumHeteroatoms(x) if x is not None else None)

# Number of hydrogen bond donors
df['num_h_donors'] = df['ROMol'].apply(lambda x: Descriptors.NumHDonors(x) if x is not None else None)

# Number of hydrogen bond acceptors
df['num_h_acceptors'] = df['ROMol'].apply(lambda x: Descriptors.NumHAcceptors(x) if x is not None else None)

# Number of rotatable bonds
df['num_rot_bonds'] = df['ROMol'].apply(lambda x: Descriptors.NumRotatableBonds(x) if x is not None else None)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11759 entries, 0 to 11763
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Formula             11759 non-null  object 
 1   FW                  11759 non-null  object 
 2   DSSTox_CID          11759 non-null  object 
 3   SR-HSE              8147 non-null   object 
 4   ID                  11759 non-null  object 
 5   ROMol               11759 non-null  object 
 6   NR-AR               9358 non-null   object 
 7   SR-ARE              7165 non-null   object 
 8   NR-Aromatase        7222 non-null   object 
 9   NR-ER-LBD           8749 non-null   object 
 10  NR-AhR              8165 non-null   object 
 11  SR-MMP              7317 non-null   object 
 12  NR-ER               7694 non-null   object 
 13  NR-PPAR-gamma       8180 non-null   object 
 14  SR-p53              8630 non-null   object 
 15  SR-ATAD5            9087 non-null   object 
 16  NR-AR-LBD

*Determine if toxicophore presents in the molecule.*

In [ ]:
# Defining toxicophore SMARTS patterns
toxicophores = {
    'aromatic_amine': '[NX3;H2;R0;!$(NC=O)]',
    'aromatic_nitroso': '[NX2]=O',
    'alkyl_nitrile': 'C#N',
    'nitrosamine': '[NX3;H2;!$(NC=O);!$(N=C=O)]',
    'sulphonate_bonded_carbon': '[CX4;H0;R0;+0]',
    'unsaturated_aldehyde': '[CX3]=[OX1]',
    'aliphatic_N_nitro': '[NX3;H2;!$(NC=O)](=O)',
    'aziridine': 'C1CN1',
    'azide': '[NX-]=[NX2+]',
    'diazo': 'N=[N+]=[N-]',
    'triazene': '[NX2]=[NX1]=[NX1]',
    'diazonium': '[NX2+]#[NX1-]',
    'beta_propiolactone': 'C#CC(=O)',
    'unsaturated_alpha_beta_unsaturated_alkoxy': '[#6]=[#6][O][#6]',
    'unsubstituted_heteroatombonded_heteroatom': '[!#6][!#6]-[!#6]',
    'aromatic_hydroxylamine': '[NX3;H2;!$(NC=O)]O',
    'aliphatic_halide': '[CX4;H0;R0;Cl,Br,I]',
    'carboxylic_acid_halide': '[CX3](=O)[F,Cl,Br,I]',
    '1_aryl_2_monoalkyl_hydrazine': '[#6]N([#6;!$(C=[N,O,S]);!$(CC)])[#6]',
    'aromatic_methylamine': 'c[NH2]',
    'aromatic_hydroxylamine_ester': '[NX3;H2;!$(NC=O)]OC',
    'polycyclic_aromatic_system': '[cR1]1[cR1][cR1][cR1][cR1][cR1]1',
    'Bay_region': '[cR1]1[cR1][cR1][cR1][cR1][cR1]1',
    'K_region': '[cR1]1[cR1][cR1][cR1][cR1][cR1]1',
    'polycyclic_planar_system': '[cR1]1[cR1][cR1][cR1][cR1][cR1]1'
}

# Creating a function to check for the presence of toxicophores in a molecule
def has_toxicophore(mol, toxicophore):
    if mol is None:
        return False
    return mol.HasSubstructMatch(Chem.MolFromSmarts(toxicophores[toxicophore]))

# Creating a copy of the DataFrame
df_copy = df.copy()

# Checking for the presence of each toxicophore in each molecule and adding columns for them
for toxicophore in toxicophores.keys():
    df_copy[f'has_{toxicophore}_toxicophore'] = df_copy['ROMol'].apply(lambda x: has_toxicophore(x, toxicophore))

# Displaying the copied DataFrame with the new columns
df_copy.head()

,Formula,FW,DSSTox_CID,SR-HSE,ID,ROMol,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,...,has_aromatic_hydroxylamine_toxicophore,has_aliphatic_halide_toxicophore,has_carboxylic_acid_halide_toxicophore,has_1_aryl_2_monoalkyl_hydrazine_toxicophore,has_aromatic_methylamine_toxicophore,has_aromatic_hydroxylamine_ester_toxicophore,has_polycyclic_aromatic_system_toxicophore,has_Bay_region_toxicophore,has_K_region_toxicophore,has_polycyclic_planar_system_toxicophore
0,C27H25ClN6,468.9806 (35.4535+224.2805+209.2465),25848,0,NCGC00178831-03,<rdkit.Chem.rdchem.Mol object at 0x7a5c14e78ac0>,NaN,NaN,NaN,NaN,...,False,False,False,False,True,False,False,False,False,False
1,C20H6Br4Na2O5,691.8542 (645.8757+22.9892+22.9892),5234,0,NCGC00166114-03,<rdkit.Chem.rdchem.Mol object at 0x7a5c19365460>,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,True,True,True,True
2,C47H83NO17,934.1584 (916.1205+18.0379),28909,0,NCGC00263563-01,<rdkit.Chem.rdchem.Mol object at 0x7a5c120feea0>,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,C52H54N4O12,927.0048 (329.4575+89.0275+89.0275+329.4575+90...,5513,1,NCGC00013058-02,<rdkit.Chem.rdchem.Mol object at 0x7a5c193c70d0>,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,True,True,True,True
4,C66H87N17O14,1342.5025 (1282.4505+60.0520),26683,NaN,NCGC00167516-01,<rdkit.Chem.rdchem.Mol object at 0x7a5c1200b680>,0,NaN,NaN,NaN,...,False,False,False,False,False,False,True,True,True,True


In [ ]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11759 entries, 0 to 11763
Data columns (total 51 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Formula                                                    11759 non-null  object 
 1   FW                                                         11759 non-null  object 
 2   DSSTox_CID                                                 11759 non-null  object 
 3   SR-HSE                                                     8147 non-null   object 
 4   ID                                                         11759 non-null  object 
 5   ROMol                                                      11759 non-null  object 
 6   NR-AR                                                      9358 non-null   object 
 7   SR-ARE                                                     7165 non-null   object 
 8   NR-Aromatas

*Vector representation of molecular structures.*

In [ ]:
#Downloading model_300dim.pkl
url = 'https://github.com/samoturk/mol2vec_notebooks/raw/master/Notebooks/model_300dim.pkl'
file_to_download = requests.get(url, allow_redirects=True)

open('model_300dim.pkl', 'wb').write(file_to_download.content)

26567327

In [ ]:
#Loading pre-trained model via word2vec
model = gensim.models.Word2Vec.load('model_300dim.pkl')

In [ ]:
#Constructing sentences
df['sentence'] = df['ROMol'].apply(lambda x: MolSentence(mol2alt_sentence(x, 1)) if x is not None else None)

In [ ]:
#Extracting embeddings to a numpy.array
def sentences2vec(sentences, model, unseen=None):
    keys = set(model.wv.key_to_index)  # Access key_to_index through wv
    vec = []
    for sentence in sentences:
        this_vec = []
        for word in sentence:
            if word in keys:
                this_vec.append(model.wv[word])  # Access vectors through wv
            elif unseen:
                this_vec.append(model.wv[unseen])
        if this_vec:
            vec.append(np.mean(this_vec, axis=0))
        else:
            vec.append(np.zeros(model.vector_size))
    return vec

df['mol2vec'] = sentences2vec(df['sentence'], model, unseen='UNK') #Mark unseen='UNK' for model to handle unknown substructures

# Converting embeddings to a suitable format
mol2vec_features = np.array(df['mol2vec'].tolist())
mol2vec_df = pd.DataFrame(mol2vec_features, columns=[f'mol2vec_{i}' for i in range(mol2vec_features.shape[1])])

# Merging with the original DataFrame
df = pd.concat([df, mol2vec_df], axis=1)

df.head()

,Formula,FW,DSSTox_CID,SR-HSE,ID,ROMol,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,...,mol2vec_90,mol2vec_91,mol2vec_92,mol2vec_93,mol2vec_94,mol2vec_95,mol2vec_96,mol2vec_97,mol2vec_98,mol2vec_99
0,C27H25ClN6,468.9806 (35.4535+224.2805+209.2465),25848,0,NCGC00178831-03,<rdkit.Chem.rdchem.Mol object at 0x7df76e150f20>,NaN,NaN,NaN,NaN,...,0.236166,0.157575,-0.162339,-0.362618,0.273598,-0.540641,0.078816,0.326415,-0.105661,-0.049595
1,C20H6Br4Na2O5,691.8542 (645.8757+22.9892+22.9892),5234,0,NCGC00166114-03,<rdkit.Chem.rdchem.Mol object at 0x7df76e150cf0>,NaN,NaN,NaN,NaN,...,0.180145,0.173445,-0.156019,-0.282326,0.170189,-0.550380,0.011285,0.330538,-0.176390,-0.087855
2,C47H83NO17,934.1584 (916.1205+18.0379),28909,0,NCGC00263563-01,<rdkit.Chem.rdchem.Mol object at 0x7df76f47c510>,NaN,NaN,NaN,NaN,...,0.435524,0.024827,-0.139884,-0.339033,0.602306,-0.348037,0.237839,0.053543,-0.441683,0.066746
3,C52H54N4O12,927.0048 (329.4575+89.0275+89.0275+329.4575+90...,5513,1,NCGC00013058-02,<rdkit.Chem.rdchem.Mol object at 0x7df76f47c900>,NaN,NaN,NaN,NaN,...,0.378559,0.029953,-0.102872,-0.256238,0.231509,-0.537275,0.163485,0.372507,-0.282478,-0.073305
4,C66H87N17O14,1342.5025 (1282.4505+60.0520),26683,NaN,NCGC00167516-01,<rdkit.Chem.rdchem.Mol object at 0x7df76f47c970>,0,NaN,NaN,NaN,...,0.341454,-0.068446,0.058678,-0.348879,0.141241,-0.447340,0.189109,0.367658,-0.473593,-0.006741


In [ ]:
#Calculating NaN values for each column
nan_count_per_column = df.isna().sum()
print("\nNumber of NaN values in each column:")
print(nan_count_per_column)


Number of NaN values in each column:
Formula          5
FW               5
DSSTox_CID       5
SR-HSE        3617
ID               5
              ... 
mol2vec_95       5
mol2vec_96       5
mol2vec_97       5
mol2vec_98       5
mol2vec_99       5
Length: 128, dtype: int64


In [ ]:
# Saving the features to a CSV file
df.to_csv('/content/drive/My Drive/df.csv', index=False)